# Семинар 3. Исправление опечаток

# Домашняя работа, Игнатьев Д.

## Алгоритм symspell

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter


In [106]:

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def preprocess(text):
    sents = sentenize(text)
    return [normalize(sent.text) for sent in sents]

def ngrammer(tokens, n):
    ngrams = []
    tokens = [token for token in tokens]
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [6]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
file = open('data/correct_sents.txt', encoding='utf8').read()
true = file.splitlines()

In [3]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

### рекурсивная функция для удаления символов

In [4]:
def edits(word, n_edits=1):
    spellings = []
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes = [L + R[1:]               for L, R in splits if R]
    spellings += deletes
    if n_edits == 1:
        return spellings
    else:
        for n in deletes:
            spellings += edits(n, n_edits-1)
        return spellings

In [8]:
probs_list = re.findall('\w+', corpus.lower())
probs = Counter(vocab_list)

In [10]:
# фунцкия расчета вероятности слова
N = sum(probs.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return probs[word] / N

In [12]:
P("смешарики")

3.879518450892764e-07

In [9]:
vocab_list = re.findall('\w+', file.lower())
print(len(vocab_list))

10177


### Генерация словаря вариантов

In [82]:
# создаем словарь
vocab = {n:n for n in vocab_list}

for word in vocab_list:
    options = edits(word, n_edits=2)
    for n in options:
        if n not in vocab:
            vocab[n] = word
        else:
            # Проверка, что слово не было ранее добавлено через запятую
            if re.compile(word).search(vocab[n]) == None:
                vocab[n] += f", {word}"

### Функции исправления

In [83]:
def corrections(word, vocab=vocab, n_edits=1):
    corrected = []
    options = edits(word, n_edits=n_edits)
    if word in vocab and vocab[word] == word:
        return [word]
    for n in options:
        if n in vocab:
            corrected += vocab[n].split(", ")
        
    if len(corrected) == 0:
        return [word]
    return list(set(corrected))

In [84]:
corrections('молох', n_edits=2)

['холод',
 'колой',
 'волос',
 'мало',
 'моего',
 'много',
 'масло',
 'мол',
 'моих',
 'молока',
 'голос',
 'блоха',
 'полно',
 'мылом',
 'можно',
 'долго',
 'плохо',
 'около',
 'молчу',
 'обоих',
 'могла']

### Проверка работы алгоритма

In [85]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [89]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        predicted = cashed.get(pair[1], max(corrections(pair[1], n_edits=1), key=P))
        cashed[pair[0]] = predicted
        
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1
        
    if not i % 100:
        print(i)
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0
100
200
300
400
500
600
700
800
900
0.586913086913087
0.5587106676899463
0.40886642930975076


### При расстоянии редактирования 2 в словаре охват ошибок выше половины, но значительно число исправлений корректных слов.
### При расстоянии редактирования 1 в словаре меньше неправильных исправлений (20%), но доля исправленных ошибок снижается до 30%.

## Исправления с триграммной моделью

In [90]:
corpus_wiki = [['<start>', '<start>'] + sent + ['<end>'] for sent in preprocess(corpus)]

In [91]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [95]:
unigrams = Counter()
bigrams = Counter()
trigrams = Counter()

for sentence in corpus_wiki:
    unigrams.update(sentence)
    bigrams.update(ngrammer(sentence))
    trigrams.update(ngrammer(sentence, n=3))

In [115]:
# оцените качество также как и раньше
def predict_mistaken(word, vocab):
    if word in vocab:
        return 0
    else:
        return 1

mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    word_pairs = [('<start>', '<start>')] + word_pairs
    pred_sent = []
    for j in range(1, len(word_pairs)):
        
        pred = None
        
        # проверяем, что слова нет в словаре, чтобы не исправлять все слова
        if not predict_mistaken(word_pairs[j][1], probs):
            pred = word_pairs[j][1]
            
        
        else:
            # находим кандидатов для исправления
            predicted = corrections(pair[1], n_edits=1)
            #get_closest_hybrid_match(word_pairs[j][1], X, vec)
        
            # берем биграм для контекста
            if j == 1:
                preceding_bigram = '<start> <start>'
            else:
                preceding_bigram = word_pairs[j-2][0] + " " + word_pairs[j-1][0]
#             prev_word = word_pairs[j-1][1]
        
            # если у нас нет в модели такого биграмма
            if preceding_bigram not in bigrams:
                pred = max(corrections(word_pairs[j][1], n_edits=1), key=P)
        
            else:
                #
                lm_predicted = []
                for word in predicted:
                    # поскольку нет возможности учитывать косинусную близость, будем умножать на вероятность самого слова
                    m = P(word)
                    trigram = preceding_bigram + " " + word
                    # домножаем полученную метрику для слова на вероятность триграма
                    # триграм - предыдущий биграм + текущее слово кандидат
                    lm_predicted.append((word, (m)*((trigrams[trigram]/bigrams[preceding_bigram])))) 
                
                
                if lm_predicted:
                    pred = sorted(lm_predicted, key=lambda x: -x[1])[0][0]
        
        if pred is None:
            pred = word_pairs[j][1]

        
        if pred == word_pairs[j][0]:
            correct += 1
        else:
            mistakes.append((word_pairs[j][0], word_pairs[j][1], pred))
        total += 1
            
        if word_pairs[j][0] == word_pairs[j][1]:
            total_correct += 1
            if word_pairs[j][0] !=  pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if word_pairs[j][0] == pred:
                mistaken_fixed += 1
    
    if not i % 100:
        print(i)

0
100
200
300
400
500
600
700
800
900


In [116]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8524475524475524
0.12125863392171911
0.03813024003675204


### Комментарий: добавление триграммной модели сильно минимизировало процент ненужных исправлений. При этом часть нужных исправлений также отпала, поскольку вероятность соответствующих биграмм оказалась небольшой